In [10]:
# Import statements

import numpy as np
import pandas as pd

from sklearn.model_selection import KFold, cross_val_score, train_test_split # Imported performance/cross-validation
from sklearn.linear_model import SGDClassifier # Imported model

In [2]:
# Reading into data frame from file

data_frame = pd.read_csv('./creditcard.csv')

In [3]:
# Dataset Validation

# Validation the dataset isn't malformed and list features
# data_frame.head()
# data_frame.columns

# Exploration
# data_frame.shape

In [4]:
# Data Preprocessing

# Check if data has null values
# print(data_frame.isnull().sum())
# Check if data has zero values
# print((data_frame == 0).sum())

In [12]:
# Data to fit and targets

feature_set = ["Time", "V1", "V2", "V3", "V4", "V5", "V6", "V7", "V8", "V9", "V10", "V11", "V12", "V13", "V14", "V15", "V16", "V17", "V18", "V19", "V20", "V21", "V22", "V23", "V24", "V25", "V26", "V27", "V28", "Amount"]
target_set = ["Class"]
X = data_frame[feature_set]
y = data_frame[target_set]
# print(X)
# print(y)

In [6]:
# Initialize model(s)

model = SGDClassifier(random_state=1)

In [ ]:
# Need a way to scale the data so we aren't so unbalanced

In [13]:
# Split data and do cross validation

# Splits data into 80-20 train-test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1)

# Does cross validation with 5 folds using our training data post split
kf = KFold(n_splits=5)
cv_scores = cross_val_score(model, X_train, y_train, cv=kf, scoring="accuracy")
print("Cross validation scores:", cv_scores)
print(f"Mean Validation Score: {np.mean(cv_scores)}")

/home/quetzl/.pyenv/versions/ml-project-2025/lib/python3.12/site-packages/sklearn/utils/validation.py:1406: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/home/quetzl/.pyenv/versions/ml-project-2025/lib/python3.12/site-packages/sklearn/utils/validation.py:1406: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/home/quetzl/.pyenv/versions/ml-project-2025/lib/python3.12/site-packages/sklearn/utils/validation.py:1406: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/home/quetzl/.pyenv/versions/ml-project-2025/lib/python3.12/site-packages/sklearn/utils/validation.py:1406: D

Cross validation scores: [0.99828831 0.9983322  0.99771775 0.99837609 0.99800303]
Mean Validation Score: 0.9981434747306283
